In [1]:
"""
This script uses the UNCOMTRADE
API to retrieve international trade
data into a Pandas DataFrame
Author: Carlos Góes
www.carlosgoes.com
"""

def comtrade(reporter,partner,year,freq,commoditycode):
    #Import your libraries
    import json
    import urllib
    import pandas as pd
    import ssl
    
    #Import the index of countries
    partnerurl = 'https://comtrade.un.org/data/cache/partnerAreas.json'
    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE
    areas = urllib.request.urlopen(partnerurl, context=ctx)
    data = json.loads(areas.read())
    data = data['results']
    index = {}
    for i in range(len(data)):
        upper = data[i]['text']
        lower = upper.lower()
        index.update({lower: data[i]['id']})
    
    #Retrieve numeric codes for reporter and partner
    reporter = index[str(reporter)]
    partner =  index[str(partner)]
    
    #Set the URL API
    url = 'http://comtrade.un.org/api/get?' + \
        'max=100000&' + \
        'type=C&' + \
        'freq=' + str(freq) + '&' + \
        'px=HS&' + \
        'ps=' + str(year) + '&' + \
        'r=' + reporter + '&' + \
        'p=' + partner + '&' + \
        'rg=all&' + \
        'cc=' + commoditycode + '&' + \
        'fmt=json'
    
    #Import data with the API, transform JSON into a frame
    urlopen = urllib.request.urlopen(url, context=ctx)
    data = json.loads(urlopen.read())
    data = pd.io.json.json_normalize(data['dataset'])
    
    #Return the data
    return data

In [2]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

PATH = r'C:\Users\Carlos\OneDrive - UC San Diego\UCSD\Research\Trade, Ideas and Productivity\code'

#Import data from 2000 through 2014
year = 2010
country = 'usa'
partner = 'brazil'
freq = 'A'
ccode = 'all'

frame = comtrade(country, partner, year, freq, ccode)

In [3]:
frame = frame[ (frame.rgCode == 2) & (frame.aggrLevel == 6)  ]
miniframe = frame[[ 'cmdDescE', 'qtCode', 'qtDesc', 'qtAltCode',
       'qtAltDesc', 'TradeQuantity', 'AltQuantity', 'NetWeight', 'GrossWeight',
       'TradeValue' ]]

In [4]:
miniframe

,cmdDescE,qtCode,qtDesc,qtAltCode,qtAltDesc,TradeQuantity,AltQuantity,NetWeight,GrossWeight,TradeValue
3449,Live horses/asses/mules/hinnies: pure-bred bre...,5,Number of items,None,,41.0,None,NaN,None,380800
3450,Live horses/asses/mules/hinnies other than pur...,5,Number of items,None,,555.0,None,NaN,None,2907262
3452,Live swine: pure-bred breeding animals,5,Number of items,None,,1456.0,None,108775.0,None,948380
3454,"Live fowls of species Gallus domesticus, weigh...",5,Number of items,None,,325940.0,None,13898.0,None,893054
3456,"Live mammals, n.e.s.",5,Number of items,None,,730.0,None,351.0,None,23538
...,...,...,...,...,...,...,...,...,...,...
8344,"Original sculptures & statuary, in any material",1,No Quantity,None,,NaN,None,NaN,None,4780181
8346,"Postage/revenue stamps, stamp-postmarks, first...",1,No Quantity,None,,NaN,None,NaN,None,3500
8348,Collections & collectors' pieces of zoological...,1,No Quantity,None,,NaN,None,NaN,None,18295
8350,Antiques of an age > one hundred years,1,No Quantity,None,,NaN,None,NaN,None,1162732
